In [2]:
import pandas as pd
import json
from pandas import json_normalize
import os
import warnings
import glob
from tqdm import tqdm

warnings.filterwarnings("ignore")
output_dir = 'output'

In [8]:
with open('input/group-data/data-part1.json') as f:
    data = json.load(f)


In [27]:
# Initialize empty DataFrames
members_df = pd.DataFrame()
admins_df = pd.DataFrame()
former_members_df = pd.DataFrame()
feed_df = pd.DataFrame()

# Process each group in the JSON
for group_id, group_data in data.items():
    
    # Process members using json_normalize
    if 'members' in group_data and isinstance(group_data['members'], list) and any(isinstance(item, dict) for item in group_data['members']):
        members = [item for item in group_data['members'] if isinstance(item, dict) and 'id' in item]
        if members:
            group_members_df = json_normalize(members)
            group_members_df['group_id'] = group_id
            group_members_df['member_type'] = 'current'
            members_df = pd.concat([members_df, group_members_df], ignore_index=True)
    
    # Process admins using json_normalize
    if 'admins' in group_data and isinstance(group_data['admins'], list) and any(isinstance(item, dict) for item in group_data['admins']):
        admins = [item for item in group_data['admins'] if isinstance(item, dict) and 'id' in item]
        if admins:
            group_admins_df = json_normalize(admins)
            group_admins_df['group_id'] = group_id
            group_admins_df['member_type'] = 'admin'
            admins_df = pd.concat([admins_df, group_admins_df], ignore_index=True)
    
    # Process former members using json_normalize
    if 'former_members' in group_data and isinstance(group_data['former_members'], list) and any(isinstance(item, dict) for item in group_data['former_members']):
        former = [item for item in group_data['former_members'] if isinstance(item, dict) and 'id' in item]
        if former:
            group_former_df = json_normalize(former)
            group_former_df['group_id'] = group_id
            group_former_df['member_type'] = 'former'
            former_members_df = pd.concat([former_members_df, group_former_df], ignore_index=True)
    
    # Process feed items with special handling for properties field
    if 'feed' in group_data and isinstance(group_data['feed'], list) and any(isinstance(item, dict) for item in group_data['feed']):
        feed_items = [item for item in group_data['feed'] if isinstance(item, dict) and 'id' in item]
        
        # Pre-process feed items to handle the properties field
        for item in feed_items:
            if 'properties' in item and isinstance(item['properties'], list) and len(item['properties']) > 0:
                # Extract the first (and typically only) properties item
                props = item['properties'][0]
                if isinstance(props, dict):
                    # Convert properties to flat fields
                    for key, value in props.items():
                        item[f'properties_{key}'] = value
                # Remove the original properties list
                del item['properties']
                
        if feed_items:
            # Use json_normalize
            group_feed_df = json_normalize(feed_items, sep='_')
            group_feed_df['group_id'] = group_id
            feed_df = pd.concat([feed_df, group_feed_df], ignore_index=True)

# Combine all members, admins, and former members
all_members_df = pd.concat([members_df, admins_df, former_members_df], ignore_index=True)

# Save to CSV files - no printing headers or concatenating all data
if not all_members_df.empty:
    writer = pd.ExcelWriter(
        os.path.join(output_dir, 'group_members.xlsx'), 
        engine='xlsxwriter',
        engine_kwargs={'options': {'strings_to_urls': False}}  # Prevent URL detection
    )
    all_members_df.to_excel(writer, index=False)
    writer.close()
    
    # Save pickle file for lossless data storage
    all_members_df.to_pickle(os.path.join(output_dir, 'group_members.pkl'))

if not feed_df.empty:
    writer = pd.ExcelWriter(
        os.path.join(output_dir, 'group_feed.xlsx'), 
        engine='xlsxwriter',
        engine_kwargs={'options': {'strings_to_urls': False}}  # Prevent URL detection
    )
    feed_df.to_excel(writer, index=False)
    writer.close()
    
    # Save pickle file for lossless data storage
    feed_df.to_pickle(os.path.join(output_dir, 'group_feed.pkl'))

In [4]:
def process_json_data(json_data):
    """
    Process JSON data and extract members and feed information into DataFrames.
    
    Args:
        json_data (dict): The JSON data to process
        
    Returns:
        tuple: (members_df, feed_df) - DataFrames containing member and feed data
    """
    # Initialize empty DataFrames
    members_df = pd.DataFrame()
    admins_df = pd.DataFrame()
    former_members_df = pd.DataFrame()
    feed_df = pd.DataFrame()

    # Process each group in the JSON
    for group_id, group_data in json_data.items():
        
        # Process members using json_normalize
        if 'members' in group_data and isinstance(group_data['members'], list) and any(isinstance(item, dict) for item in group_data['members']):
            members = [item for item in group_data['members'] if isinstance(item, dict) and 'id' in item]
            if members:
                group_members_df = json_normalize(members)
                group_members_df['group_id'] = group_id
                group_members_df['member_type'] = 'current'
                members_df = pd.concat([members_df, group_members_df], ignore_index=True)
        
        # Process admins using json_normalize
        if 'admins' in group_data and isinstance(group_data['admins'], list) and any(isinstance(item, dict) for item in group_data['admins']):
            admins = [item for item in group_data['admins'] if isinstance(item, dict) and 'id' in item]
            if admins:
                group_admins_df = json_normalize(admins)
                group_admins_df['group_id'] = group_id
                group_admins_df['member_type'] = 'admin'
                admins_df = pd.concat([admins_df, group_admins_df], ignore_index=True)
        
        # Process former members using json_normalize
        if 'former_members' in group_data and isinstance(group_data['former_members'], list) and any(isinstance(item, dict) for item in group_data['former_members']):
            former = [item for item in group_data['former_members'] if isinstance(item, dict) and 'id' in item]
            if former:
                group_former_df = json_normalize(former)
                group_former_df['group_id'] = group_id
                group_former_df['member_type'] = 'former'
                former_members_df = pd.concat([former_members_df, group_former_df], ignore_index=True)
        
        # Process feed items with special handling for properties field
        if 'feed' in group_data and isinstance(group_data['feed'], list) and any(isinstance(item, dict) for item in group_data['feed']):
            feed_items = [item for item in group_data['feed'] if isinstance(item, dict) and 'id' in item]
            
            # Pre-process feed items to handle the properties field
            for item in feed_items:
                if 'properties' in item and isinstance(item['properties'], list) and len(item['properties']) > 0:
                    # Extract the first (and typically only) properties item
                    props = item['properties'][0]
                    if isinstance(props, dict):
                        # Convert properties to flat fields
                        for key, value in props.items():
                            item[f'properties_{key}'] = value
                    # Remove the original properties list
                    del item['properties']
                    
            if feed_items:
                # Use json_normalize
                group_feed_df = json_normalize(feed_items, sep='_')
                group_feed_df['group_id'] = group_id
                feed_df = pd.concat([feed_df, group_feed_df], ignore_index=True)

    # Combine all members, admins, and former members
    all_members_df = pd.concat([members_df, admins_df, former_members_df], ignore_index=True)
    
    return all_members_df, feed_df

def process_directory(input_dir, output_dir):
    """
    Process all JSON files in a directory, extract data, and save combined results.
    
    Args:
        input_dir (str): Directory containing JSON files
        output_dir (str): Directory to save output files
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Find all JSON files in the input directory
    json_files = glob.glob(os.path.join(input_dir, "*.json"))
    
    # Initialize empty DataFrames to store combined results
    combined_members_df = pd.DataFrame()
    combined_feed_df = pd.DataFrame()
    
    # Process each JSON file with a progress bar
    for json_file in tqdm(json_files, desc="Processing JSON files"):
        try:
            # Read the JSON file
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            # Process the JSON data
            members_df, feed_df = process_json_data(data)
            
            # Add file source information
            file_name = os.path.basename(json_file)
            if not members_df.empty:
                members_df['source_file'] = file_name
            if not feed_df.empty:
                feed_df['source_file'] = file_name
            
            # Concatenate with combined results
            combined_members_df = pd.concat([combined_members_df, members_df], ignore_index=True)
            combined_feed_df = pd.concat([combined_feed_df, feed_df], ignore_index=True)
            
        except Exception as e:
            print(f"Error processing file {json_file}: {str(e)}")

    
    # Save combined results to Excel and pickle files
    if not combined_members_df.empty:
        # Save to Excel with URL detection disabled
        # writer = pd.ExcelWriter(
        #     os.path.join(output_dir, 'combined_members.xlsx'), 
        #     engine='xlsxwriter',
        #     engine_kwargs={'options': {'strings_to_urls': False}}
        # )
        # combined_members_df.to_excel(writer, index=False)
        # writer.close()
        combined_members_df.to_csv(os.path.join(output_dir, 'combined_members.csv'), index=False)
        # Save to pickle
        combined_members_df.to_pickle(os.path.join(output_dir, 'combined_members.pkl'))
        
        print(f"Saved combined members data: {len(combined_members_df)} records")
    
    if not combined_feed_df.empty:
        # Save to Excel with URL detection disabled
        # writer = pd.ExcelWriter(
        #     os.path.join(output_dir, 'combined_feed.xlsx'), 
        #     engine='xlsxwriter',
        #     engine_kwargs={'options': {'strings_to_urls': False}}
        # )
        # combined_feed_df.to_excel(writer, index=False)
        # writer.close()
        combined_feed_df.to_csv(os.path.join(output_dir, 'combined_feed.csv'), index=False)
        # Save to pickle
        combined_feed_df.to_pickle(os.path.join(output_dir, 'combined_feed.pkl'))
        
        print(f"Saved combined feed data: {len(combined_feed_df)} records")

In [11]:
process_directory('input/group-data', 'output')

Processing JSON files: 100%|██████████| 9/9 [00:33<00:00,  3.67s/it]


Saved combined members data: 2265807 records
Saved combined feed data: 390505 records


In [12]:
combined_members_df = pd.read_pickle('output/combined_members.pkl')
combined_feed_df = pd.read_pickle('output/combined_feed.pkl')

In [14]:
combined_members_df.describe(include='all')

,id,first_name,last_name,title,department,primary_address,account_invite_time,about,group_id,member_type,source_file
count,2265807,2265807,2265807,2070958,2145491,2138348,2215035,22190,2265807,2265807,2265807
unique,103137,28222,42902,1867,516,1503,22731,422,1241,3,9
top,100011662284821,Daniel,Singh,Checkout Operator,Checkouts,9841 Favona Rd - Support Office,2018-03-03T09:13:46+0000,Kia ora! I'm part of the Internal Communicatio...,164920673960046,former,data-part6.json
freq,770,10668,83997,338350,334577,65072,4205,563,66425,1613125,529704


In [15]:
dtypes = {
    'id': 'string',  # Using string for ID (appears to be a large number that may not need arithmetic)
    'first_name': 'category',  # Category for names since there are only 25,041 unique values
    'last_name': 'category',   # Category for names since there are only 38,158 unique values
    'title': 'category',       # Only 1,853 unique values
    'department': 'category',  # Only 513 unique values
    'primary_address': 'category',  # Only 1,500 unique values
    'about': 'string',         # Free text field with relatively few entries
    'group_id': 'category',    # Only 1,241 unique values
    'member_type': 'category', # Only 3 unique values
    'source_file': 'category'  # Only 4 unique values
}

# Convert data types
combined_members_df = combined_members_df.astype(dtypes)
combined_members_df['account_invite_time'] = pd.to_datetime(combined_members_df['account_invite_time']).dt.tz_localize(None)

In [23]:
combined_members_df[combined_members_df['id']== "61566568671372"]

,id,first_name,last_name,title,department,primary_address,account_invite_time,about,group_id,member_type,source_file
43690,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23 17:17:17,<NA>,202034830884367,current,data-part5.json
246283,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23 17:17:17,<NA>,478660545673726,current,data-part9.json
321594,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23 17:17:17,<NA>,669757193210650,current,data-part8.json
360977,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23 17:17:17,<NA>,601840116664451,current,data-part8.json
380012,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23 17:17:17,<NA>,494245640766660,current,data-part8.json
399194,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23 17:17:17,<NA>,487190284822117,current,data-part8.json
660748,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23 17:17:17,<NA>,1050338732866843,current,data-part4.json
679489,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23 17:17:17,<NA>,2328007430742432,current,data-part4.json
727076,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23 17:17:17,<NA>,477379202696755,current,data-part4.json
1143774,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23 17:17:17,<NA>,980229266005433,current,data-part1.json


In [17]:
dtypes = {
    'id': 'string',  # All unique values, maintain as string
    'icon': 'category',  # Only 4 unique values
    'is_hidden': 'boolean',  # Boolean type (shows only 1 unique value: False)
    'link': 'string',  # Many unique values
    'message': 'string',  # Free text field
    'object_id': 'string',  # Numeric-looking IDs stored as strings
    'permalink_url': 'string',  # All unique URLs
    'picture': 'string',  # URLs for pictures
    'source': 'string',  # Source URLs
    'properties_name': 'category',  # Only 3 unique values
    'properties_text': 'string',  # Mixed text content
    'from_name': 'category',  # Name with moderate cardinality
    'from_id': 'category',  # ID with moderate cardinality
    'privacy_allow': 'category',  # Only 1 unique value
    'privacy_deny': 'category',  # Only 1 unique value
    'privacy_description': 'category',  # Only 1 unique value
    'privacy_friends': 'category',  # Only 1 unique value
    'privacy_value': 'category',  # Only 1 unique value
    'message_tags': 'string',  # Could be JSON-like content
    'caption': 'string',  # Text captions
    'name': 'string',  # Names with high cardinality
    'description': 'string',  # Text descriptions
    'shares_count': 'Int64',  # Nullable integer type
    'group_id': 'category',  # Moderate cardinality
    'parent_id': 'string',  # Parent IDs
    'place_name': 'category',  # Location names
    'place_location_city': 'category',  # City names
    'place_location_country': 'category',  # Country names (only 7 unique)
    'place_location_latitude': 'float64',  # Geographic coordinates
    'place_location_longitude': 'float64',  # Geographic coordinates
    'place_id': 'category',  # Place IDs
    'place_location_street': 'category',  # Street names
    'place_location_zip': 'category',  # Zip codes
    'place_location_state': 'category',  # State names (only 3 unique)
    'application_link': 'category',  # Application links
    'application_name': 'category',  # Application names (only 4 unique)
    'application_id': 'category',  # Application IDs (only 4 unique)
    'source_file': 'category',  # Source files (only 4 unique)
    'admin_creator_link': 'category',  # Admin creator links
    'admin_creator_name': 'category',  # Admin creator names
    'admin_creator_id': 'category',  # Admin creator IDs
    'place_location_located_in': 'category',  # Only 1 unique value
    'admin_creator_category': 'category',  # Only 1 unique value
    'application_category': 'category'  # Only 1 unique value
}

# Convert data types
combined_feed_df = combined_feed_df.astype(dtypes)
combined_feed_df['created_time'] = pd.to_datetime(combined_feed_df['created_time']).dt.tz_localize(None)
combined_feed_df['updated_time'] = pd.to_datetime(combined_feed_df['updated_time']).dt.tz_localize(None)


In [18]:
combined_feed_df = combined_feed_df.drop_duplicates()
combined_members_df = combined_members_df.drop_duplicates()

combined_members_df.to_pickle('output/combined_members.pkl')
combined_feed_df.to_pickle('output/combined_feed.pkl')

combined_feed_df.to_csv('output/combined_feed.csv', index=False)
combined_members_df.to_csv('output/combined_members.csv', index=False)

# Save to Excel with URL detection disabled
# writer = pd.ExcelWriter(
#     os.path.join(output_dir, 'combined_members.xlsx'), 
#     engine='xlsxwriter',
#     engine_kwargs={'options': {'strings_to_urls': False}}
# )
# combined_members_df.to_excel(writer, index=False)
# writer.close()

# # Save to pickle
# combined_members_df.to_pickle(os.path.join(output_dir, 'combined_members.pkl'))

# # Save to Excel with URL detection disabled
# writer = pd.ExcelWriter(
#     os.path.join(output_dir, 'combined_feed.xlsx'), 
#     engine='xlsxwriter',
#     engine_kwargs={'options': {'strings_to_urls': False}}
# )
# combined_feed_df.to_excel(writer, index=False)
# writer.close()

In [24]:
combined_feed_df.describe(include='all')

,id,created_time,icon,is_hidden,link,message,object_id,permalink_url,picture,updated_time,...,source_file,application_link,application_name,application_id,place_location_located_in,admin_creator_link,admin_creator_name,admin_creator_id,admin_creator_category,application_category
count,390505,390505,258123,390505,309943,348441,285822,390505,307875,390505,...,390505,151,151,151,2,87,87,87,40,40
unique,390505,NaN,4,1,266348,332451,252321,390505,299678,NaN,...,9,4,4,4,2,3,3,3,1,1
top,1588681094756721_3558254291132715,NaN,https://www.facebook.com/images/icons/photo.gif,False,https://workplace.metastatus.com/,Morning grade Bridge street,1168470823551649,https://countdown.facebook.com/groups/15886810...,https://static.xx.fbcdn.net/rsrc.php/v4/yJ/r/9...,NaN,...,data-part3.json,https://countdown.workplace.com/1944452369100565,Enablo Tools,117350288946782,145444595465627,https://countdown.workplace.com/175825029867996,WoW Nominate,791937734337394,Utilities,Utilities
freq,1,NaN,243023,390505,292,445,68,1,857,NaN,...,61712,63,63,63,1,45,45,45,40,40
mean,NaN,2020-03-08 23:10:15.861476608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-12 14:40:32.808017920,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,2016-04-18 06:35:54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-18 06:35:54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,2018-05-16 01:54:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-05-18 03:55:04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,2019-10-24 07:22:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-29 21:01:34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,2021-12-15 23:25:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-21 01:22:33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,2025-03-17 19:35:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-18 02:03:55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
groups_only_data_path = 'input/groups.json'
with open(groups_only_data_path) as f:
    groups_only_data = json.load(f)
groups_only_df = pd.json_normalize(groups_only_data)
groups_only_df

,id,is_workplace_default,name,privacy,updated_time,is_community,description,owner.name,owner.id
0,843031417379447,False,Opx Specialists,CLOSED,2024-07-23T02:20:31+0000,False,NaN,NaN,NaN
1,434116296111923,False,Avonhead VMB,OPEN,2024-07-01T20:34:11+0000,False,Daily updates,Andrew Hinman,100013749646759
2,1408000779918059,False,Store Planner & Customer Offer,SECRET,2024-08-20T21:06:33+0000,False,NaN,Caleb Tawa,100090419557642
3,1472025353519797,False,PRODUCE WAIATA,CLOSED,2024-06-20T08:57:37+0000,False,NaN,Ashwin Prasad,100091289592257
4,3601957096692302,False,Dump Huddles 9460,CLOSED,2024-05-16T00:32:41+0000,False,NaN,Caleb Tawa,100090419557642
...,...,...,...,...,...,...,...,...,...
1236,1446355662143509,False,Workplace Customer Advisory Board - APAC,CLOSED,2023-07-22T11:40:04+0000,False,This is an invite-only group for the Customer ...,NaN,NaN
1237,712065508983368,False,Sei Mani Adoption Community,CLOSED,2019-04-11T08:18:33+0000,False,The purpose of this community is to help peopl...,NaN,NaN
1238,916725538491714,False,St Lukes,SECRET,2018-03-02T07:51:30+0000,False,Work with people from other companies on St Lu...,NaN,NaN
1239,2010263422586265,False,Workplace Login with Access Code Beta Test,SECRET,2024-05-29T04:00:10+0000,False,NaN,NaN,NaN


In [50]:
members_only_data_path = 'input/members.json'
with open(members_only_data_path) as f:
    members_only_data = json.load(f)
members_only_df = pd.json_normalize(members_only_data["members"])
members_only_df

,id,first_name,last_name,title,department,primary_address,account_invite_time,about
0,61565475637447,Ezekiel,Nakaroti,Nightfill Assistant,NZ Long-Life Night,Ferrymead,2024-09-09T16:32:37+0000,NaN
1,61565537547717,Taylor,Erskine,Nightfill Assistant,NZ Long-Life Night,Invercargill,2024-09-05T23:14:04+0000,NaN
2,61566568671372,Morgan,Glover,Nightfill Assistant,NZ Long-Life Night,Hamilton,2024-09-23T17:17:17+0000,NaN
3,100087527932748,Macy,Breingan,Supervisor - Online,NZ On-Line,Whangaparaoa,2022-11-14T22:31:15+0000,NaN
4,61565692348357,Francesca,Aquino,Online Assistant,NZ On-Line,Whangarei,2024-09-09T16:46:55+0000,NaN
...,...,...,...,...,...,...,...,...
19257,100013512616589,Losaline,Hudson,Assistant Customer Service Manager,NZ Checkouts,9472CDPapamoa,2019-11-18T12:25:54+0000,NaN
19258,100013572336054,Grace,Goyon,Customer Service Manager,NZ Management Admin,Onehunga,2019-11-18T12:33:42+0000,NaN
19259,100012874862688,Rhonda,Imrie,Supervisor - Checkouts,NZ Checkouts,9157CDUpper Hutt,2019-11-18T18:55:38+0000,NaN
19260,100012895770084,Tania,Walker,Supervisor - Checkouts,NZ Checkouts,9456CDTaupo,2019-11-18T12:54:46+0000,NaN


In [52]:
members_only_data.keys()

dict_keys(['members', 'admins', 'former_members'])

In [53]:
len(members_only_data['members']), len(members_only_data['admins']), len(members_only_data['former_members'])

(19262, 6, 47303)

In [14]:
def process_json_group_data(json_data, groups_only_df, group_lookup=None):
    """
    Process JSON data and extract members IDs, feed IDs, and feed information.
    
    Args:
        json_data (dict): The JSON data to process
        groups_only_df (DataFrame): DataFrame containing group information with 'id' column
        group_lookup (dict, optional): Existing lookup dictionary for group data
        
    Returns:
        tuple: (updated_groups_df, feed_df, group_lookup) - DataFrames and updated lookup
    """
    # Initialize empty feed DataFrame
    feed_df = pd.DataFrame()
    
    # Use provided lookup or create a new one if not provided
    if group_lookup is None:
        # Create a copy of the groups_only_df to avoid modifying the original
        groups_df = groups_only_df.copy()
        
        # Create columns for the member lists if they don't exist
        for col in ['current_members', 'admin_members', 'former_members', 'feed_ids']:
            if col not in groups_df.columns:
                groups_df[col] = None
        
        # Create a lookup dictionary for faster access (O(1) instead of O(n))
        group_lookup = groups_df.set_index('id').to_dict(orient='index')
    
    # Process each group in the JSON
    for group_id, group_data in json_data.items():
        
        # Extract member IDs
        current_members = []
        admin_members = []
        former_members = []
        feed_ids = []
        
        # Process current members
        if 'members' in group_data and isinstance(group_data['members'], list):
            current_members = [item['id'] for item in group_data['members'] 
                              if isinstance(item, dict) and 'id' in item]
            
        # Process admin members
        if 'admins' in group_data and isinstance(group_data['admins'], list):
            admin_members = [item['id'] for item in group_data['admins'] 
                            if isinstance(item, dict) and 'id' in item]
            
        # Process former members
        if 'former_members' in group_data and isinstance(group_data['former_members'], list):
            former_members = [item['id'] for item in group_data['former_members'] 
                             if isinstance(item, dict) and 'id' in item]
        
        # Process feed items
        if 'feed' in group_data and isinstance(group_data['feed'], list):
            feed_items = [item for item in group_data['feed'] 
                         if isinstance(item, dict) and 'id' in item]
            
            # Extract feed IDs
            feed_ids = [item['id'] for item in feed_items if 'id' in item]
            
            # Pre-process feed items to handle the properties field
            for item in feed_items:
                if 'properties' in item and isinstance(item['properties'], list) and len(item['properties']) > 0:
                    # Extract the first (and typically only) properties item
                    props = item['properties'][0]
                    if isinstance(props, dict):
                        # Convert properties to flat fields
                        for key, value in props.items():
                            item[f'properties_{key}'] = value
                    # Remove the original properties list
                    del item['properties']
                    
            if feed_items:
                # Use json_normalize
                group_feed_df = pd.json_normalize(feed_items, sep='_')
                group_feed_df['group_id'] = group_id
                feed_df = pd.concat([feed_df, group_feed_df], ignore_index=True)
        
        # Update the group information in the lookup dictionary
        if group_id in group_lookup:
            group_lookup[group_id]['current_members'] = current_members or None
            group_lookup[group_id]['admin_members'] = admin_members or None
            group_lookup[group_id]['former_members'] = former_members or None
            group_lookup[group_id]['feed_ids'] = feed_ids or None
        else:
            # Create a new entry if the group is not in the groups_only_df
            group_lookup[group_id] = {
                'current_members': current_members or None,
                'admin_members': admin_members or None,
                'former_members': former_members or None,
                'feed_ids': feed_ids or None
            }

    # Convert the lookup dictionary back to a DataFrame
    updated_groups_df = pd.DataFrame.from_dict(group_lookup, orient='index').reset_index()
    updated_groups_df.rename(columns={'index': 'id'}, inplace=True)  # Rename to 'id' to match original
    
    return updated_groups_df, feed_df, group_lookup

In [15]:
group_lookup = None
updated_groups_df, feed_df, group_lookup = process_json_group_data(data, groups_only_df, group_lookup)

In [16]:
updated_groups_df[updated_groups_df['current_members'].notnull()]

,id,is_workplace_default,name,privacy,updated_time,is_community,description,owner.name,owner.id,current_members,admin_members,former_members,feed_ids
0,843031417379447,False,Opx Specialists,CLOSED,2024-07-23T02:20:31+0000,False,NaN,NaN,NaN,[100072845286952],[100072845286952],"[100025721226334, 100012874500685, 10001270944...","[843031417379447_843048817377707, 843031417379..."
1,434116296111923,False,Avonhead VMB,OPEN,2024-07-01T20:34:11+0000,False,Daily updates,Andrew Hinman,100013749646759,"[100077979562015, 100078889187079, 10004080385...",[100013749646759],[100013896884252],"[434116296111923_434689349387951, 434116296111..."
2,1408000779918059,False,Store Planner & Customer Offer,SECRET,2024-08-20T21:06:33+0000,False,NaN,Caleb Tawa,100090419557642,"[100012722530067, 100087810461686, 10006395283...","[100012804726428, 100012746771340, 10009041955...",[100065314866311],"[1408000779918059_1443678306350306, 1408000779..."
3,1472025353519797,False,PRODUCE WAIATA,CLOSED,2024-06-20T08:57:37+0000,False,NaN,Ashwin Prasad,100091289592257,"[100091283874417, 100085547483656, 10004118893...",[100091289592257],[100014112905978],[1472025353519797_1472025393519793]
4,3601957096692302,False,Dump Huddles 9460,CLOSED,2024-05-16T00:32:41+0000,False,NaN,Caleb Tawa,100090419557642,"[100083376217814, 100012906038177, 10006392669...",[100090419557642],[100087810461686],[3601957096692302_3601957123358966]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,194474644302657,False,Takapuna Store Group,CLOSED,2018-08-25T06:24:10+0000,False,NaN,Daniel Chasemore,100011662284821,"[100013649979110, 100013582541615, 10002980456...","[100013535915514, 100011662284821]","[100026845571972, 100013606242519, 10001371402...","[194474644302657_581194215630696, 194474644302..."
1067,173627443066810,False,Northcote Store Group,CLOSED,2023-08-30T08:34:57+0000,False,NaN,Daniel Chasemore,100011662284821,"[61566646542809, 100043924352906, 615665385932...","[100013464761425, 100011662284821]","[100086342170709, 100043763471027, 10008301896...","[173627443066810_1854524838310387, 17362744306..."
1069,1097685310302250,False,Mairangi bay Store Group,CLOSED,2022-09-05T01:57:17+0000,False,NaN,Daniel Chasemore,100011662284821,"[100013489632236, 61563168954569, 615615925640...","[100013555959806, 100011662284821]","[100032521980364, 100040668079723, 10003127760...","[1097685310302250_7890129127724467, 1097685310..."
1070,1279713542048652,False,Hauraki Store Group,CLOSED,2017-09-21T09:01:30+0000,False,NaN,Daniel Chasemore,100011662284821,"[100013617303840, 100013583084623, 10001656699...","[100013563093358, 100011662284821]","[100013687427229, 100012661842548, 10001387965...","[1279713542048652_1676238342396168, 1279713542..."


In [17]:
feed_df.to_csv('output/feed_data.csv', index=False)
updated_groups_df.to_csv('output/group_data.csv', index=False)

In [18]:
updated_groups_df

,id,is_workplace_default,name,privacy,updated_time,is_community,description,owner.name,owner.id,current_members,admin_members,former_members,feed_ids
0,843031417379447,False,Opx Specialists,CLOSED,2024-07-23T02:20:31+0000,False,NaN,NaN,NaN,[100072845286952],[100072845286952],"[100025721226334, 100012874500685, 10001270944...","[843031417379447_843048817377707, 843031417379..."
1,434116296111923,False,Avonhead VMB,OPEN,2024-07-01T20:34:11+0000,False,Daily updates,Andrew Hinman,100013749646759,"[100077979562015, 100078889187079, 10004080385...",[100013749646759],[100013896884252],"[434116296111923_434689349387951, 434116296111..."
2,1408000779918059,False,Store Planner & Customer Offer,SECRET,2024-08-20T21:06:33+0000,False,NaN,Caleb Tawa,100090419557642,"[100012722530067, 100087810461686, 10006395283...","[100012804726428, 100012746771340, 10009041955...",[100065314866311],"[1408000779918059_1443678306350306, 1408000779..."
3,1472025353519797,False,PRODUCE WAIATA,CLOSED,2024-06-20T08:57:37+0000,False,NaN,Ashwin Prasad,100091289592257,"[100091283874417, 100085547483656, 10004118893...",[100091289592257],[100014112905978],[1472025353519797_1472025393519793]
4,3601957096692302,False,Dump Huddles 9460,CLOSED,2024-05-16T00:32:41+0000,False,NaN,Caleb Tawa,100090419557642,"[100083376217814, 100012906038177, 10006392669...",[100090419557642],[100087810461686],[3601957096692302_3601957123358966]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,1446355662143509,False,Workplace Customer Advisory Board - APAC,CLOSED,2023-07-22T11:40:04+0000,False,This is an invite-only group for the Customer ...,NaN,NaN,None,None,None,None
1237,712065508983368,False,Sei Mani Adoption Community,CLOSED,2019-04-11T08:18:33+0000,False,The purpose of this community is to help peopl...,NaN,NaN,None,None,None,None
1238,916725538491714,False,St Lukes,SECRET,2018-03-02T07:51:30+0000,False,Work with people from other companies on St Lu...,NaN,NaN,None,None,None,None
1239,2010263422586265,False,Workplace Login with Access Code Beta Test,SECRET,2024-05-29T04:00:10+0000,False,NaN,NaN,NaN,None,None,None,None


In [ ]:
def process_json_member_conversation_data(json_data, member_only_df, member_lookup=None):
    """
    Process JSON data for member conversations and extract message information.
    
    Args:
        json_data (dict): The JSON data to process with members and their conversations
        member_only_df (DataFrame): DataFrame containing member information with 'id' column
        member_lookup (dict, optional): Existing lookup dictionary for member data
        
    Returns:
        tuple: (updated_members_df, messages_df, member_lookup) - DataFrames and updated lookup
    """
    # Initialize empty messages DataFrame
    messages_df = pd.DataFrame()
    
    # Use provided lookup or create a new one if not provided
    if member_lookup is None:
        # Create a copy of the member_only_df to avoid modifying the original
        members_df = member_only_df.copy()
        
        # Create column for conversation ids if it doesn't exist
        for col in ['conversation_ids', 'conversation_ids_in_member_data', 'feed_ids', 'manager_ids', 'group_ids']:
            if col not in members_df.columns:
                members_df[col] = None
        
        # Create a lookup dictionary for faster access (O(1) instead of O(n))
        member_lookup = members_df.set_index('id').to_dict(orient='index')
    
    # Process each member in the JSON
    for member_id, member_data in json_data.items():
        # Initialize list to store conversation IDs for this member
        conversation_ids = []
        
        # Process each conversation
        for conversation_id, messages in member_data.items():
            # Add conversation ID to the list
            conversation_ids.append(conversation_id)
            
            # Process messages if they exist and are in a list
            if isinstance(messages, list) and len(messages) > 0:
                # Process message tags
                for message in messages:
                    # Handle tags if present
                    if 'tags' in message and isinstance(message['tags'], dict) and 'data' in message['tags']:
                        # Extract tag names into a list
                        tag_data = message['tags']['data']
                        if isinstance(tag_data, list):
                            tag_names = [item.get('name') for item in tag_data if isinstance(item, dict) and 'name' in item]
                            message['tag_names'] = tag_names
                        # Remove the original tags structure
                        del message['tags']
                
                # Normalize the messages to a DataFrame
                try:
                    conversation_messages_df = pd.json_normalize(messages, sep='_')
                    
                    # Add member and conversation IDs
                    conversation_messages_df['member_id'] = member_id
                    conversation_messages_df['conversation_id'] = conversation_id
                    
                    # Extract community ID if it exists
                    if 'from_community_id' in conversation_messages_df.columns:
                        # Already flattened by json_normalize
                        pass
                    elif any('from.community.id' in col for col in conversation_messages_df.columns):
                        # Rename the column to a simpler format
                        cols_to_rename = {col: 'from_community_id' 
                                          for col in conversation_messages_df.columns 
                                          if col == 'from.community.id'}
                        conversation_messages_df.rename(columns=cols_to_rename, inplace=True)
                    
                    # Append to the messages DataFrame
                    messages_df = pd.concat([messages_df, conversation_messages_df], ignore_index=True)
                except Exception as e:
                    print(f"Error processing conversation {conversation_id} for member {member_id}: {e}")
        
        # Update the member information in the lookup dictionary
        if member_id in member_lookup:
            member_lookup[member_id]['conversation_ids'] = conversation_ids or None
        else:
            # Create a new entry if the member is not in the member_only_df
            member_lookup[member_id] = {
                'conversation_ids': conversation_ids or None,
                'conversation_ids_in_member_data': None,
                'feed_ids': None,
                'manager_ids': None,
                'group_ids': None
            }

    # Convert the lookup dictionary back to a DataFrame
    updated_members_df = pd.DataFrame.from_dict(member_lookup, orient='index').reset_index()
    updated_members_df.rename(columns={'index': 'id'}, inplace=True)
    
    return updated_members_df, messages_df, member_lookup

In [20]:
member_lookup = None
with open('input/member-data/conversation-messages/data-part1.json') as f:
    conversation_data = json.load(f)
updated_members_df, messages_df, member_lookup = process_json_member_conversation_data(conversation_data, members_only_df, member_lookup)

In [26]:
updated_members_df[updated_members_df['conversation_ids'].notnull()]

,id,first_name,last_name,title,department,primary_address,account_invite_time,about,conversation_ids
8,61567040880197,Gurkaram,Singh,Nightfill Assistant,NZ Long-Life Night,Newmarket,2024-10-10T19:00:22+0000,NaN,[t_3045581772126155]
178,100067565379472,Thomas,Hamilton,Assistant Long Life Manager - Day,NZ Long-Life Day,Bureta Park,2021-08-03T03:04:24+0000,NaN,"[t_814249157503923, t_624515616477279, t_14341..."
179,100048916730704,Duncan,Edmonds,Warehouse Operator,Order Pulling - AM,9700 ARDC - Mangere,2020-03-25T04:39:30+0000,NaN,"[t_450049329968954, t_276889720618250, t_12623..."
180,61566564315383,Ashna,Varghese,Nightfill Assistant,NZ Long-Life Night,Newmarket,2024-10-09T16:01:51+0000,NaN,[t_3045581772126155]
189,61560478834906,Brandon,Swanson,Service Deli Manager,NZ Delicatessen,Sydenham,2024-06-05T01:03:11+0000,NaN,"[t_122121646646349294, t_122123670872349294, t..."
...,...,...,...,...,...,...,...,...,...
728,61556698942794,Sarah,Arnesen,Duty Supervior,NZ Management Admin,9469CDGreymouth,2024-02-15T23:47:14+0000,NaN,"[t_7694738640622354, t_7797044767008801]"
729,100040809521878,Vai,Tauli,Nightfill Assistant,NZ Long-Life Night,Palmerston North,2019-11-18T12:50:16+0000,NaN,"[t_637602990943346, t_469174481119532, t_23575..."
730,61550611952234,Eden,Connelly,Bakery Apprentice Tier 1,NZ Bakery,Hastings,2023-08-23T18:39:31+0000,NaN,"[t_122187614378020398, t_122164255274020398, t..."
734,100040596833394,Ngarie,Southall,Supervisor - Fresh,NZ Perishables,Marton,2019-11-18T12:55:06+0000,NaN,"[t_643442223685666, t_464413294921894, t_39914..."


In [ ]:
def process_json_member_data(json_data, member_only_df, member_lookup=None):
    """
    Process JSON data for member information, including feeds, conversations, managers, and groups.
    
    Args:
        json_data (dict): The JSON data to process with member information
        member_only_df (DataFrame): DataFrame containing member information with 'id' column
        member_lookup (dict, optional): Existing lookup dictionary for member data
        
    Returns:
        tuple: (updated_members_df, member_feeds_df, conversations_df, member_lookup)
    """
    # Initialize empty DataFrames
    member_feeds_df = pd.DataFrame()
    conversations_df = pd.DataFrame()
    
    # Use provided lookup or create a new one if not provided
    if member_lookup is None:
        # Create a copy of the member_only_df to avoid modifying the original
        members_df = member_only_df.copy()
        
        # Create columns if they don't exist
        for col in ['conversation_ids', 'conversation_ids_in_member_data', 'feed_ids', 'manager_ids', 'group_ids']:
            if col not in members_df.columns:
                members_df[col] = None
        
        # Create a lookup dictionary for faster access (O(1) instead of O(n))
        member_lookup = members_df.set_index('id').to_dict(orient='index')
    
    # Process each member in the JSON
    for member_id, member_data in json_data.items():
        # Initialize lists to store IDs
        feed_ids = []
        conversation_ids = []
        manager_ids = []
        group_ids = []
        
        # Process feed items
        if 'feed' in member_data and isinstance(member_data['feed'], list):
            feed_items = [item for item in member_data['feed'] 
                         if isinstance(item, dict) and 'id' in item]
            
            # Extract feed IDs
            feed_ids = [item['id'] for item in feed_items if 'id' in item]
            
            # Pre-process feed items to handle any nested structures if needed
            for item in feed_items:
                if 'properties' in item and isinstance(item['properties'], list) and len(item['properties']) > 0:
                    # Extract the first (and typically only) properties item
                    props = item['properties'][0]
                    if isinstance(props, dict):
                        # Convert properties to flat fields
                        for key, value in props.items():
                            item[f'properties_{key}'] = value
                    # Remove the original properties list
                    del item['properties']
                    
            if feed_items:
                # Use json_normalize
                member_feed_df = pd.json_normalize(feed_items, sep='_')
                member_feed_df['member_id'] = member_id  # Associate with member instead of group
                member_feeds_df = pd.concat([member_feeds_df, member_feed_df], ignore_index=True)
        
        # Process conversations
        if 'conversations' in member_data and isinstance(member_data['conversations'], list):
            conversations = [conv for conv in member_data['conversations'] 
                            if isinstance(conv, dict) and 'id' in conv]
            
            # Extract conversation IDs
            conversation_ids = [conv['id'] for conv in conversations if 'id' in conv]
            
            # Process each conversation
            for conv in conversations:
                # Extract participant IDs if they exist
                participant_ids = []
                if ('participants' in conv and isinstance(conv['participants'], dict) and 
                    'data' in conv['participants'] and isinstance(conv['participants']['data'], list)):
                    participant_ids = [p.get('id') for p in conv['participants']['data'] 
                                      if isinstance(p, dict) and 'id' in p]
                
                # Create a clean conversation record
                conversation_record = {
                    'id': conv.get('id'),
                    'member_id': member_id,
                    'participant_ids': participant_ids,
                    'message_count': conv.get('message_count'),
                    'unread_count': conv.get('unread_count'),
                    'updated_time': conv.get('updated_time')
                }
                
                # Add to conversations DataFrame
                conversations_df = pd.concat([conversations_df, 
                                             pd.DataFrame([conversation_record])], 
                                            ignore_index=True)
        
        # Process managers
        if 'managers' in member_data and isinstance(member_data['managers'], list):
            managers = [m for m in member_data['managers'] 
                       if isinstance(m, dict) and 'id' in m]
            manager_ids = [m['id'] for m in managers if 'id' in m]
        
        # Process groups
        if 'groups' in member_data and isinstance(member_data['groups'], list):
            groups = [g for g in member_data['groups'] 
                     if isinstance(g, dict) and 'id' in g]
            group_ids = [g['id'] for g in groups if 'id' in g]
        
        # Update the member information in the lookup dictionary
        if member_id in member_lookup:
            member_lookup[member_id]['conversation_ids_in_member_data'] = conversation_ids or None
            member_lookup[member_id]['feed_ids'] = feed_ids or None
            member_lookup[member_id]['manager_ids'] = manager_ids or None
            member_lookup[member_id]['group_ids'] = group_ids or None
        else:
            # Create a new entry if the member is not in the member_only_df
            member_lookup[member_id] = {
                'conversation_ids': None,  # Initialize this too
                'conversation_ids_in_member_data': conversation_ids or None,
                'feed_ids': feed_ids or None,
                'manager_ids': manager_ids or None,
                'group_ids': group_ids or None
            }

    # Convert the lookup dictionary back to a DataFrame
    updated_members_df = pd.DataFrame.from_dict(member_lookup, orient='index').reset_index()
    updated_members_df.rename(columns={'index': 'id'}, inplace=True)
    
    return updated_members_df, member_feeds_df, conversations_df, member_lookup

In [36]:
with open('input/member-data/data-part1.json') as f:
    data = json.load(f)

updated_members_df, member_feeds_df, conversations_df, member_lookup = process_json_member_data(data, members_only_df, member_lookup)

In [37]:
updated_members_df[updated_members_df['conversation_ids_in_member_data'].notnull()]

,id,first_name,last_name,title,department,primary_address,account_invite_time,about,conversation_ids,conversation_ids_in_member_data,feed_ids,manager_ids,group_ids
8,61567040880197,Gurkaram,Singh,Nightfill Assistant,NZ Long-Life Night,Newmarket,2024-10-10T19:00:22+0000,NaN,[t_3045581772126155],[t_3045581772126155],None,NaN,"[434701117380982, 170017273430642, 23280074307..."
175,100014529674306,Tham,Tran,Category Assistant,Management & Supervision,9690 Byron St - Support Office,2018-12-28T00:18:52+0000,NaN,None,"[t_1358924961268480, t_1330317630795880, t_131...","[100014529674306_3405420573075215, 10001452967...",NaN,"[594840900935712, 2328007430742432, 2310220762..."
178,100067565379472,Thomas,Hamilton,Assistant Long Life Manager - Day,NZ Long-Life Day,Bureta Park,2021-08-03T03:04:24+0000,NaN,"[t_814249157503923, t_624515616477279, t_14341...","[t_6244904595596908, t_7723166927696433, t_796...","[100067565379472_1703583206760444, 10006756537...",NaN,"[966915016768865, 767645407418738, 75941111119..."
179,100048916730704,Duncan,Edmonds,Warehouse Operator,Order Pulling - AM,9700 ARDC - Mangere,2020-03-25T04:39:30+0000,NaN,"[t_450049329968954, t_276889720618250, t_12623...","[t_450049329968954, t_276889720618250, t_12623...",[100048916730704_1967532940365468],NaN,"[706703882860938, 2328007430742432, 2310220762..."
180,61566564315383,Ashna,Varghese,Nightfill Assistant,NZ Long-Life Night,Newmarket,2024-10-09T16:01:51+0000,NaN,[t_3045581772126155],[t_3045581772126155],None,NaN,"[2328007430742432, 2310220762632001, 224074831..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8433,61554426955275,George,Warnakulasurlya,Supervisor - Checkouts,Checkouts,Mt Roskill,2023-12-05T13:39:03+0000,NaN,None,"[t_122134487834147565, t_122134474442147565, t...",None,NaN,"[1761708497434048, 2328007430742432, 175344694..."
8435,61553943044097,Jelena,Mako,Checkout Operator,NZ Checkouts,Fraser Cove,2023-12-04T20:56:27+0000,NaN,None,"[t_5913699332017842, t_6240015286076593, t_122...",None,NaN,"[2328007430742432, 1050338732866843, 224952101..."
8437,61553996411383,Ma Lourdes,Gatus,Checkout Operator,NZ Checkouts,Pokeno,2023-12-04T20:16:57+0000,NaN,None,[t_7701723109847340],None,NaN,"[671496463856221, 2328007430742432, 1050338732..."
8458,61554144786030,James,Maskell,Fresh Food Assistant,NZ Produce,Tauranga,2023-12-04T16:57:35+0000,NaN,None,[t_6827802800671894],None,NaN,"[2328007430742432, 1050338732866843, 224952101..."


In [38]:
member_feeds_df.to_csv('output/member_feeds.csv', index=False)
conversations_df.to_csv('output/conversations.csv', index=False)

In [12]:
messages_df[messages_df['conversation_id'] == 't_364249529590951']

,id,created_time,message,tag_names,from_id,from_name,shares_data,shares_paging_cursors_before,shares_paging_cursors_after,shares_paging_next,member_id,conversation_id,from_community_id,attachments_data,attachments_paging_cursors_before,attachments_paging_cursors_after,attachments_paging_next,from_email
140582,m_JK7dqYs4dcpWTll69Ojt0GjIOWDoBltaJKiMnWVBl-mD...,2023-12-12T04:30:11+0000,Hi Trudy I put this on our checkout group the ...,"[inbox, messenger, read, source:mobile]",100084168462491,Andrea Craig,NaN,NaN,NaN,NaN,100080177206492,t_364249529590951,229949227363012,NaN,NaN,NaN,NaN,NaN


In [24]:
members_only_df[members_only_df['conversation_ids_in_member_data'].isnull()]['conversation_ids_in_member_data']

2        None
5        None
6        None
7        None
8        None
         ... 
66519    None
66521    None
66527    None
66539    None
66557    None
Name: conversation_ids_in_member_data, Length: 21768, dtype: object

In [65]:
def extract_data(data_dir):
    """
    Extract and process data from a directory structure containing JSON files.
    
    Args:
        data_dir (str): Path to the directory containing the data
        
    Returns:
        tuple: (groups_df, members_df, group_feeds_df, member_feeds_df, 
                conversations_df, messages_df) - Processed DataFrames
    """
    # Initialize empty DataFrames for collecting results
    all_group_feeds_df = pd.DataFrame()
    all_member_feeds_df = pd.DataFrame()
    all_conversations_df = pd.DataFrame()
    all_messages_df = pd.DataFrame()
    
    # Path to groups and members JSON files
    groups_only_data_path = os.path.join(data_dir, 'groups.json')
    members_only_data_path = os.path.join(data_dir, 'members.json')
    
    # Read and process groups data
    print("Reading groups data...")
    with open(groups_only_data_path) as f:
        groups_only_data = json.load(f)
    groups_only_df = pd.json_normalize(groups_only_data)
    
    # Read and process members data
    print("Reading members data...")
    with open(members_only_data_path) as f:
        members_only_data = json.load(f)

    # Process each member type separately and add the member_type column
    members_dfs = []

    # Process admin members first (highest priority)
    if "admins" in members_only_data and isinstance(members_only_data["admins"], list):
        admin_members_df = pd.json_normalize(members_only_data["admins"])
        admin_members_df["member_type"] = "admin"
        members_dfs.append(admin_members_df)
        # Keep track of admin IDs to filter out duplicates later
        admin_ids = set(admin_members_df['id'].tolist())
    else:
        admin_ids = set()

    # Process current members (excluding those who are admins)
    if "members" in members_only_data and isinstance(members_only_data["members"], list):
        current_members_df = pd.json_normalize(members_only_data["members"])
        # Filter out members who are also admins
        if admin_ids:
            current_members_df = current_members_df[~current_members_df['id'].isin(admin_ids)]
        current_members_df["member_type"] = "current"
        members_dfs.append(current_members_df)
        # Update set of processed IDs
        processed_ids = admin_ids.union(set(current_members_df['id'].tolist()))
    else:
        processed_ids = admin_ids

    # Process former members (excluding duplicates)
    if "former_members" in members_only_data and isinstance(members_only_data["former_members"], list):
        former_members_df = pd.json_normalize(members_only_data["former_members"])
        # Filter out already processed members
        if processed_ids:
            former_members_df = former_members_df[~former_members_df['id'].isin(processed_ids)]
        former_members_df["member_type"] = "former"
        members_dfs.append(former_members_df)

    # Combine all member DataFrames
    members_only_df = pd.concat(members_dfs, ignore_index=True) if members_dfs else pd.DataFrame()
        
    # Create lookup dictionaries
    group_lookup = None
    member_lookup = None
    
    # Find all JSON files in the directory, excluding groups.json and members.json
    all_json_files = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.json') and file not in ['groups.json', 'members.json']:
                all_json_files.append(os.path.join(root, file))
    
    print(f"Found {len(all_json_files)} JSON files to process.")
    
    # Create a progress bar for processing files
    pbar = tqdm(all_json_files, desc="Processing files")
    
    # Process each JSON file based on its directory name
    for file_path in pbar:
        # Get the parent directory name
        parent_dir = os.path.basename(os.path.dirname(file_path))
        file_name = os.path.basename(file_path)
        
        # For nested folders, get the parent's parent directory
        if parent_dir == 'conversation-messages':
            grand_parent_dir = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
            if grand_parent_dir == 'member-data':
                parent_dir = 'conversation-messages'
        
        # Update progress bar description with current file and its folder
        pbar.set_description(f"Processing {parent_dir}/{file_name}")
        
        # Load the JSON file
        with open(file_path) as f:
            json_data = json.load(f)
        
        # Process based on the directory name
        if parent_dir == 'group-data':
            # Process group data
            updated_groups_df, group_feeds_df, group_lookup = process_json_group_data(
                json_data, groups_only_df, group_lookup
            )
            
            # Update groups DataFrame for next iteration
            groups_only_df = updated_groups_df
            
            # Accumulate feed data
            if not group_feeds_df.empty:
                all_group_feeds_df = pd.concat([all_group_feeds_df, group_feeds_df], ignore_index=True)
                
        elif parent_dir == 'member-data':
            # Process member data
            updated_members_df, member_feeds_df, conversations_df, member_lookup = process_json_member_data(
                json_data, members_only_df, member_lookup
            )
            
            # Update members DataFrame for next iteration
            members_only_df = updated_members_df
            
            # Accumulate feed and conversation data
            if not member_feeds_df.empty:
                all_member_feeds_df = pd.concat([all_member_feeds_df, member_feeds_df], ignore_index=True)
            
            if not conversations_df.empty:
                all_conversations_df = pd.concat([all_conversations_df, conversations_df], ignore_index=True)
                
        elif parent_dir == 'conversation-messages':
            # Process conversation messages
            updated_members_df, messages_df, member_lookup = process_json_member_conversation_data(
                json_data, members_only_df, member_lookup
            )
            
            # Update members DataFrame for next iteration
            members_only_df = updated_members_df
            
            # Accumulate message data
            if not messages_df.empty:
                all_messages_df = pd.concat([all_messages_df, messages_df], ignore_index=True)
        
        else:
            # Update progress bar with skipped file
            pbar.set_description(f"Skipping {parent_dir}/{file_name} - unknown directory type")
    
    # Close the progress bar
    pbar.close()
    
    print("Processing complete!")

    output_dir = data_dir + '_converted'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    print(f"Saving data to {output_dir}...")

    # Save all DataFrames to CSV files
    groups_only_df.to_csv(os.path.join(output_dir, 'groups.csv'), index=False)
    members_only_df.to_csv(os.path.join(output_dir, 'members.csv'), index=False)
    all_group_feeds_df.to_csv(os.path.join(output_dir, 'group_feeds.csv'), index=False)
    all_member_feeds_df.to_csv(os.path.join(output_dir, 'member_feeds.csv'), index=False)
    all_conversations_df.to_csv(os.path.join(output_dir, 'conversations.csv'), index=False)
    all_messages_df.to_csv(os.path.join(output_dir, 'member_messages.csv'), index=False)

    # Save all DataFrames to PKL files
    groups_only_df.to_pickle(os.path.join(output_dir, 'groups.pkl'))
    members_only_df.to_pickle(os.path.join(output_dir, 'members.pkl'))
    all_group_feeds_df.to_pickle(os.path.join(output_dir, 'group_feeds.pkl'))
    all_member_feeds_df.to_pickle(os.path.join(output_dir, 'member_feeds.pkl'))
    all_conversations_df.to_pickle(os.path.join(output_dir, 'conversations.pkl'))
    all_messages_df.to_pickle(os.path.join(output_dir, 'member_messages.pkl'))

    print(f"Data successfully saved to {output_dir}!")
    
    # Return all the processed DataFrames
    return (
        groups_only_df,           # Updated groups DataFrame
        members_only_df,          # Updated members DataFrame
        all_group_feeds_df,       # Group feeds
        all_member_feeds_df,      # Member feeds
        all_conversations_df,     # Conversations
        all_messages_df           # Messages
    )

In [66]:
groups_only_df, members_only_df, group_feeds_df, member_feeds_df, conversations_df, messages_df = extract_data('input')

Reading groups data...
Reading members data...
Found 29 JSON files to process.


Processing group-data/data-part6.json: 100%|██████████| 29/29 [16:49<00:00, 34.82s/it]            


Processing complete!
Saving data to input_converted...
Data successfully saved to input_converted!


In [5]:
# groups_only_df = pd.read_pickle('input_converted/groups.pkl')
members_only_df = pd.read_pickle('input_converted/members.pkl')
group_feeds_df = pd.read_pickle('input_converted/group_feeds.pkl')
# member_feeds_df = pd.read_pickle('input_converted/member_feeds.pkl')
# conversations_df = pd.read_pickle('input_converted/conversations.pkl')
messages_df = pd.read_pickle('input_converted/member_messages.pkl')

In [11]:
total_members, n_receivers, n_senders, n_group_op =    members_only_df['id'].nunique(), messages_df['member_id'].nunique(), messages_df['from_id'].nunique(), group_feeds_df['from_id'].nunique()

In [14]:
print(f"Total number of members in Woolies data: {total_members}")
print(f"Total number of unique receivers in Woolies data: {n_receivers} ({n_receivers/total_members*100:.2f}%)")
print(f"Total number of unique senders in Woolies data: {n_senders} ({n_senders/total_members*100:.2f}%)")
print(f"Total number of unique OP in group posts data: {n_group_op} ({n_group_op/total_members*100:.2f}%)")

Total number of members in Woolies data: 66565
Total number of unique receivers in Woolies data: 919 (1.38%)
Total number of unique senders in Woolies data: 6351 (9.54%)
Total number of unique OP in group posts data: 10689 (16.06%)


In [17]:
messages_df.shape

(1659997, 18)

In [15]:
all_group_feeds_bunning_gemini = pd.concat([pd.read_parquet(f) for f in glob.glob('Bunnings_converted_parquet/group_feeds_*.parquet')])

In [16]:
all_group_feeds_bunning_gemini['from_id'].nunique()

40591

In [8]:
group_feeds_df['from_id']

0         100081242752401
1         100012739931968
2         100071045888673
3         100012666798052
4         100015210223534
               ...       
390500    100014450656641
390501    100012833674254
390502    100012833674254
390503    100012833674254
390504    100012833674254
Name: from_id, Length: 390505, dtype: object

In [12]:
members_only_df

,id,first_name,last_name,title,department,primary_address,account_invite_time,member_type,about,conversation_ids,feed_ids,manager_ids,group_ids
0,100011662284821,Daniel,Chasemore,Communication Manager,Management & Supervision,9841 Favona Rd - Support Office,2019-02-07T22:42:16+0000,admin,NaN,"[t_2211326762599378, t_2206430036422384, t_450...","[100011662284821_1984729212003327, 10001166228...",None,"[1140861446259512, 396864436572036, 2328007430..."
1,100012266170978,SNSVC0003265 StoreComms,Broker Account,NaN,C&P,NaN,2018-12-11T05:41:18+0000,admin,NaN,"[t_1671878896564300, t_617175715367962, t_9370...","[100012266170978_1571893539896170, 10001226617...",[100011662284821],"[2328007430742432, 1050338732866843, 382254555..."
2,100023940977454,Aleks,Gorgievski,"Head of Internal Comms - Platforms, Content & ...",INTERNAL COMMUNICATIONS CHAPTER,Support Location NSW - Norwest,2019-01-06T21:09:54+0000,admin,NaN,None,[100023940977454_2363574310500441],None,"[2328007430742432, 2310220762632001, 230714873..."
3,100043641968802,Enablo,Enablo,NaN,NaN,NaN,2019-11-14T01:00:20+0000,admin,NaN,"[t_402763014521761, t_109258443872221, t_16550...",None,None,"[2328007430742432, 1050338732866843, 477379202..."
4,100067146504140,Pritish,Singh,Specialist NZ - Identity Access Management,Management & Supervision,"80 Favona Road, Auckland 2024",2021-05-05T02:08:31+0000,admin,NaN,"[t_289304799965380, t_4893866934068824, t_7524...","[100067146504140_381292850785595, 100067146504...",None,"[2328007430742432, 1050338732866843, 191916723..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66560,100021265148797,Jacqueline,Jones,Store Manager,Management (Admin),Huntly Countdown,2019-11-18T12:06:52+0000,former,NaN,"[t_5509014975790246, t_3300402413422690, t_333...","[100021265148797_1978622125633657, 10002126514...",None,None
66561,100018253682633,John,Dutson-Whitter,NaN,Bakery,Newtown,2017-06-15T21:06:20+0000,former,NaN,"[t_191416818143440, t_191416981476757, t_19141...",None,None,None
66562,100017821548427,Torrie,Martinez,NaN,Checkouts,Huntly,2017-06-09T02:36:27+0000,former,NaN,"[t_114349305835787, t_417107758727010, t_32430...","[100017821548427_3225309367702531, 10001782154...",None,None
66563,100012890010510,Former,member,NaN,NaN,NaN,2019-11-18T12:22:54+0000,former,NaN,"[t_714760705623786, t_2868752579858549, t_9695...","[100012890010510_2373754989531382, 10001289001...",None,None


In [33]:
conversations_df

,id,member_id,participant_ids,message_count,unread_count,updated_time
0,t_1256270684949505,100020384897063,"[100016997551453, 100020384897063]",1,0,2023-05-23T02:15:44+0000
1,t_862516684437820,100020384897063,"[112181847962284, 100020384897063]",1,0,2021-11-29T21:21:34+0000
2,t_801748140514675,100020384897063,"[100041025360032, 100020384897063]",1,0,2021-08-17T02:34:27+0000
3,t_882551382302039,100020384897063,"[100016415553174, 100020384897063]",1,0,2021-04-21T02:44:24+0000
4,t_711217176234439,100020384897063,"[376042293235955, 100020384897063]",1,0,2021-03-04T21:31:26+0000
...,...,...,...,...,...,...
647190,t_190545434713480,100012740748257,"[100013377970185, 100012740748257]",7,0,2016-11-11T07:51:07+0000
647191,t_159468961154461,100012740748257,"[100012827075865, 100012740748257]",10,0,2016-11-10T08:01:44+0000
647192,t_1217574068285714,100012740748257,"[100012652419197, 100012740748257, 10001274890...",5,0,2016-11-09T08:46:11+0000
647193,t_1101271623281675,100012740748257,"[100012652419197, 100012690557760, 10001271872...",130,0,2016-09-03T20:43:25+0000


In [34]:
# Extract IDs from both columns into sets
conversation_ids_in_member_data_set = set()
conversation_ids_set = set(messages_df['conversation_id'].unique())

# Iterate through the dataframe to extract all IDs
for row in members_only_df.itertuples():
    # Extract IDs from conversation_ids_in_member_data
    if row.conversation_ids is not None:
        conversation_ids_in_member_data_set.update(row.conversation_ids)

# Find IDs that are common to both columns
common_ids = conversation_ids_in_member_data_set.intersection(conversation_ids_set)

# Find IDs only in the first column
only_in_member_data = conversation_ids_in_member_data_set - conversation_ids_set

# Find IDs only in the second column
only_in_conversation_ids = conversation_ids_set - conversation_ids_in_member_data_set

# Print results
print(f"Number of IDs in conversation_ids_in_member_data: {len(conversation_ids_in_member_data_set)}")
print(f"Number of IDs in conversation_ids: {len(conversation_ids_set)}")
print(f"Number of common IDs: {len(common_ids)}")
print(f"Number of IDs only in conversation_ids_in_member_data: {len(only_in_member_data)}")
print(f"Number of IDs only in conversation_ids: {len(only_in_conversation_ids)}")

# Optional: Look at some examples
print("\nSample of common IDs (up to 5):")
print(list(common_ids)[:5])

print("\nSample of IDs only in conversation_ids_in_member_data (up to 5):")
print(list(only_in_member_data)[:5])

print("\nSample of IDs only in conversation_ids (up to 5):")
print(list(only_in_conversation_ids)[:5])

Number of IDs in conversation_ids_in_member_data: 364877
Number of IDs in conversation_ids: 10867
Number of common IDs: 10867
Number of IDs only in conversation_ids_in_member_data: 354010
Number of IDs only in conversation_ids: 0

Sample of common IDs (up to 5):
['t_7487515941354418', 't_816605939173244', 't_5830064723689026', 't_128208036528826', 't_1740795643102953']

Sample of IDs only in conversation_ids_in_member_data (up to 5):
['t_1131968487237696', 't_223735186746746', 't_1557460304713633', 't_8383913215014371', 't_402018960259240']

Sample of IDs only in conversation_ids (up to 5):
[]


In [36]:
# save only in member data as json
with open('output/only_in_member_data.json', 'w') as f:
    json.dump(list(only_in_member_data), f)

In [37]:
members_only_df[members_only_df["conversation_ids"].notnull()]

,id,first_name,last_name,title,department,primary_address,account_invite_time,member_type,about,conversation_ids,feed_ids,manager_ids,group_ids
0,100011662284821,Daniel,Chasemore,Communication Manager,Management & Supervision,9841 Favona Rd - Support Office,2019-02-07T22:42:16+0000,admin,NaN,"[t_2211326762599378, t_2206430036422384, t_450...","[100011662284821_1984729212003327, 10001166228...",None,"[1140861446259512, 396864436572036, 2328007430..."
1,100012266170978,SNSVC0003265 StoreComms,Broker Account,NaN,C&P,NaN,2018-12-11T05:41:18+0000,admin,NaN,"[t_1671878896564300, t_617175715367962, t_9370...","[100012266170978_1571893539896170, 10001226617...",[100011662284821],"[2328007430742432, 1050338732866843, 382254555..."
3,100043641968802,Enablo,Enablo,NaN,NaN,NaN,2019-11-14T01:00:20+0000,admin,NaN,"[t_402763014521761, t_109258443872221, t_16550...",None,None,"[2328007430742432, 1050338732866843, 477379202..."
4,100067146504140,Pritish,Singh,Specialist NZ - Identity Access Management,Management & Supervision,"80 Favona Road, Auckland 2024",2021-05-05T02:08:31+0000,admin,NaN,"[t_289304799965380, t_4893866934068824, t_7524...","[100067146504140_381292850785595, 100067146504...",None,"[2328007430742432, 1050338732866843, 191916723..."
14,61567040880197,Gurkaram,Singh,Nightfill Assistant,NZ Long-Life Night,Newmarket,2024-10-10T19:00:22+0000,current,NaN,[t_3045581772126155],None,None,"[434701117380982, 170017273430642, 23280074307..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66560,100021265148797,Jacqueline,Jones,Store Manager,Management (Admin),Huntly Countdown,2019-11-18T12:06:52+0000,former,NaN,"[t_5509014975790246, t_3300402413422690, t_333...","[100021265148797_1978622125633657, 10002126514...",None,None
66561,100018253682633,John,Dutson-Whitter,NaN,Bakery,Newtown,2017-06-15T21:06:20+0000,former,NaN,"[t_191416818143440, t_191416981476757, t_19141...",None,None,None
66562,100017821548427,Torrie,Martinez,NaN,Checkouts,Huntly,2017-06-09T02:36:27+0000,former,NaN,"[t_114349305835787, t_417107758727010, t_32430...","[100017821548427_3225309367702531, 10001782154...",None,None
66563,100012890010510,Former,member,NaN,NaN,NaN,2019-11-18T12:22:54+0000,former,NaN,"[t_714760705623786, t_2868752579858549, t_9695...","[100012890010510_2373754989531382, 10001289001...",None,None


In [13]:
group_feeds_df

,id,created_time,icon,is_hidden,link,message,object_id,permalink_url,picture,updated_time,...,place_location_state,application_link,application_name,application_id,place_location_located_in,admin_creator_link,admin_creator_name,admin_creator_id,admin_creator_category,application_category
0,1588681094756721_3558254291132715,2024-08-30T01:56:14+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,# **NI Meat Team. Please Read**,518359230882164,https://countdown.facebook.com/groups/15886810...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2024-08-30T01:56:15+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588681094756721_3558187831139361,2024-08-29T23:30:58+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,"All discoloured, do I mark them down or just d...",1967479797019999,https://countdown.facebook.com/groups/15886810...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2024-08-30T06:32:11+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1588681094756721_3558108591147285,2024-08-29T20:34:22+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,Friday and lamb legs still coming in at incorr...,530764715968387,https://countdown.facebook.com/groups/15886810...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2024-08-30T01:39:32+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1588681094756721_3556815604609917,2024-08-28T03:30:03+0000,NaN,False,NaN,Goodafternoon team. Hey I'm having trouble sen...,NaN,https://countdown.facebook.com/groups/15886810...,NaN,2024-08-28T19:00:09+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1588681094756721_3555038108121000,2024-08-26T01:03:56+0000,NaN,False,NaN,Hi Team \n\nOur Hellers Prepacks on Monday see...,NaN,https://countdown.facebook.com/groups/15886810...,NaN,2024-08-26T05:05:40+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390500,934719143328659_947109825422924,2016-12-13T20:55:01+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,Are you an experienced Assistant Long Life Man...,125194934638861,https://countdown.facebook.com/groups/93471914...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2016-12-13T20:55:45+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390501,934719143328659_946533388813901,2016-12-13T02:48:00+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,There's lots of great things about being a par...,226665357771252,https://countdown.facebook.com/groups/93471914...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2017-01-03T22:21:28+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390502,934719143328659_935340133266560,2016-11-29T03:04:55+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,"Hi Team,\n\nDid you know as Countdown team mem...",218748918562896,https://countdown.facebook.com/groups/93471914...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2016-11-29T03:04:55+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390503,934719143328659_935294503271123,2016-11-29T01:11:35+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,NaN,218695198568268,https://countdown.facebook.com/groups/93471914...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2016-11-29T01:11:35+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
member_feeds_df

,id,created_time,icon,is_hidden,link,message,name,object_id,permalink_url,picture,...,description,place_location_zip,place_location_street,application_link,application_name,application_id,application_category,place_location_state,place_location_located_in,application_namespace
0,100020384897063_2601604460131041,2020-11-19T22:04:39+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,Updates on 3 for $20 start from Week 22\n\nTot...,Photos from Alexander Ng's post,653654768657347,https://countdown.facebook.com/groups/15886810...,https://scontent-jnb2-1.xx.fbcdn.net/v/t1.6435...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100020384897063_2598297937128360,2020-11-15T23:56:53+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,3 for $20 from Week 22\n\nAdd 2 articles as hi...,Photos from Alexander Ng's post,650919322264225,https://countdown.facebook.com/groups/15886810...,https://scontent-jnb2-1.xx.fbcdn.net/v/t1.6435...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100020384897063_2575436366081184,2020-10-21T22:51:37+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,3 for $20 updates effective from week 18\n\nCh...,Photos from Alexander Ng's post,632248424131315,https://countdown.facebook.com/groups/15886810...,https://scontent-jnb2-1.xx.fbcdn.net/v/t1.6435...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100020384897063_2572785306346290,2020-10-19T03:50:11+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,"Updated 3 for $20\nFrom this week onward, add ...",Photos from Alexander Ng's post,630088601013964,https://countdown.facebook.com/groups/15886810...,https://scontent-jnb2-1.xx.fbcdn.net/v/t1.6435...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100020384897063_2549925795298908,2020-09-22T23:49:31+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,3 for $20 - Effect Immediately National\n\nAdd...,Photos from Alexander Ng's post,612620679427423,https://countdown.facebook.com/groups/15886810...,https://scontent-jnb2-1.xx.fbcdn.net/v/t1.6435...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378769,100012740748257_1662015277454345,2016-07-22T01:24:39+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,Whoooohoooooo! 155 days to go!,NaN,134749640293060,https://countdown.facebook.com/groups/16482105...,https://scontent-jnb2-1.xx.fbcdn.net/v/t39.308...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378770,100012740748257_1661538117502061,2016-07-21T05:05:56+0000,NaN,False,NaN,Heard the store had a challanger for spruiking...,NaN,NaN,https://countdown.facebook.com/groups/16482105...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378771,100012740748257_1661344320854774,2016-07-20T19:09:04+0000,NaN,False,NaN,Loving the whole FB at work thing.......What o...,NaN,NaN,https://countdown.facebook.com/groups/16482105...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378772,100012740748257_1661094000879806,2016-07-20T08:10:00+0000,NaN,False,NaN,"So, I have had a few of our team ask me about ...",NaN,NaN,https://countdown.facebook.com/groups/16482105...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
conversations_df

,id,member_id,participant_ids,message_count,unread_count,updated_time
0,t_1256270684949505,100020384897063,"[100016997551453, 100020384897063]",1,0,2023-05-23T02:15:44+0000
1,t_862516684437820,100020384897063,"[112181847962284, 100020384897063]",1,0,2021-11-29T21:21:34+0000
2,t_801748140514675,100020384897063,"[100041025360032, 100020384897063]",1,0,2021-08-17T02:34:27+0000
3,t_882551382302039,100020384897063,"[100016415553174, 100020384897063]",1,0,2021-04-21T02:44:24+0000
4,t_711217176234439,100020384897063,"[376042293235955, 100020384897063]",1,0,2021-03-04T21:31:26+0000
...,...,...,...,...,...,...
647190,t_190545434713480,100012740748257,"[100013377970185, 100012740748257]",7,0,2016-11-11T07:51:07+0000
647191,t_159468961154461,100012740748257,"[100012827075865, 100012740748257]",10,0,2016-11-10T08:01:44+0000
647192,t_1217574068285714,100012740748257,"[100012652419197, 100012740748257, 10001274890...",5,0,2016-11-09T08:46:11+0000
647193,t_1101271623281675,100012740748257,"[100012652419197, 100012690557760, 10001271872...",130,0,2016-09-03T20:43:25+0000


In [ ]:
messages_df[messages_df['conversation_id'] == 't_189478019088405']

,id,created_time,message,tag_names,from_id,from_name,shares_data,shares_paging_cursors_before,shares_paging_cursors_after,shares_paging_next,member_id,conversation_id,from_community_id,attachments_data,attachments_paging_cursors_before,attachments_paging_cursors_after,attachments_paging_next,from_email
0,m_De2e5LbW3NMwNTrNf5fWKOQQB22mt07k-ngL3gGrUo37...,2020-02-05T04:32:04+0000,Hello again! Do you have time to answer 2 ques...,"[inbox, source:web]",346691805981916,Survey,[{'id': 'm_De2e5LbW3NMwNTrNf5fWKOQQB22mt07k-ng...,MAZDZD,MAZDZD,https://graph.facebook.com/v22.0/m_De2e5LbW3NM...,100040786243114,t_189478019088405,NaN,NaN,NaN,NaN,NaN,NaN
1,m_bRv1782fQJV4uv3cFkNAGOQQB22mt07k-ngL3gGrUo3M...,2020-02-04T04:32:03+0000,"Hi Trevor, how are you finding Workplace? Plea...","[inbox, source:web]",346691805981916,Survey,[{'id': 'm_bRv1782fQJV4uv3cFkNAGOQQB22mt07k-ng...,MAZDZD,MAZDZD,https://graph.facebook.com/v22.0/m_bRv1782fQJV...,100040786243114,t_189478019088405,NaN,NaN,NaN,NaN,NaN,NaN


: 

In [6]:
groups_only_df_memory = pd.read_pickle('input_converted_memory/groups_updated.pkl')
members_only_df_memory = pd.read_pickle('input_converted_memory/members_updated.pkl')
group_feeds_df_memory = pd.read_pickle('input_converted_memory/group_feeds.pkl')
member_feeds_df_memory = pd.read_pickle('input_converted_memory/member_feeds.pkl')
conversations_df_memory = pd.read_pickle('input_converted_memory/conversations.pkl')
messages_df_memory = pd.read_pickle('input_converted_memory/member_messages.pkl')

In [5]:
group_feeds_df_memory.describe(include='all')

,id,created_time,icon,is_hidden,link,message,object_id,permalink_url,picture,updated_time,...,application_link,application_name,application_id,place_location_state,admin_creator_link,admin_creator_name,admin_creator_id,admin_creator_category,application_category,place_location_located_in
count,390505,390505,258123,390505,309943,348441,285822,390505,307875,390505,...,151,151,151,21,87,87,87,40,40,2
unique,390505,389334,4,1,266348,332451,252321,390505,299678,389489,...,4,4,4,6,3,3,3,1,1,2
top,128195067894176_1465490687497934,2021-06-24T01:17:32+0000,https://www.facebook.com/images/icons/photo.gif,False,https://workplace.metastatus.com/,Morning grade Bridge street,1168470823551649,https://countdown.facebook.com/groups/12819506...,https://static.xx.fbcdn.net/rsrc.php/v4/yJ/r/9...,2021-06-24T01:17:36+0000,...,https://countdown.workplace.com/1944452369100565,Enablo Tools,117350288946782,QLD,https://countdown.workplace.com/175825029867996,WoW Nominate,791937734337394,Utilities,Utilities,145444595465627
freq,1,22,243023,390505,292,445,68,1,857,15,...,63,63,63,7,45,45,45,40,40,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
bunnings_dir = "Bunnings_converted"

with open(os.path.join(bunnings_dir, 'group_feeds.pkl'), 'rb') as f:
    bunnings_group_feeds_df = pd.read_pickle(f)

with open(os.path.join(bunnings_dir, 'groups.pkl'), 'rb') as f:
    bunnings_groups_df = pd.read_pickle(f)

In [4]:
bunnings_group_feeds_df

,id,created_time,is_hidden,updated_time,name,message,from_id,from_name,privacy_allow,privacy_deny,privacy_description,privacy_friends,privacy_value,group_id
0,#anonymous-user-id#5dzj2ImJ3mrvZ382mEI1ZrtFyiB...,2025-03-17T00:30:21+0000,False,2025-03-17T00:30:22+0000,Grainne,# **All Blacks Experience**\n\nLook at this am...,#anonymous-user-id#3Ts5kDycBNOfCRkKeJLjTfhfhT3...,None,,,,,CUSTOM,#anonymous-group-id#5dzj2ImJ3mrvZ382mEI1ZrtFyi...
1,#anonymous-user-id#5dzj2ImJ3mrvZ382mEI1ZrtFyiB...,2025-03-16T22:31:37+0000,False,2025-03-16T22:51:42+0000,NaN,Happy St Patricks Team,#anonymous-user-id#W5I9AV/ydQhX+Jsw/vDSaEuFwNF...,None,,,,,CUSTOM,#anonymous-group-id#5dzj2ImJ3mrvZ382mEI1ZrtFyi...
2,#anonymous-user-id#5dzj2ImJ3mrvZ382mEI1ZrtFyiB...,2025-03-13T23:45:01+0000,False,2025-03-13T23:45:02+0000,NaN,# **Don't Sleep on Safety!**\n\nHere are some ...,#anonymous-user-id#p00zuJNcojWSOgF0kQ4/WggKuaY...,None,,,,,CUSTOM,#anonymous-group-id#5dzj2ImJ3mrvZ382mEI1ZrtFyi...
3,#anonymous-user-id#5dzj2ImJ3mrvZ382mEI1ZrtFyiB...,2025-03-13T22:35:01+0000,False,2025-03-13T22:35:01+0000,NaN,# **Don't Sleep on Safety!**\n\nHere are some ...,#anonymous-user-id#p00zuJNcojWSOgF0kQ4/WggKuaY...,None,,,,,CUSTOM,#anonymous-group-id#5dzj2ImJ3mrvZ382mEI1ZrtFyi...
4,#anonymous-user-id#5dzj2ImJ3mrvZ382mEI1ZrtFyiB...,2025-03-13T20:48:16+0000,False,2025-03-13T20:48:16+0000,Photos from Caitlin Attwood's post,# **Don't Sleep on safety!**\n\nSleep is very ...,#anonymous-user-id#p00zuJNcojWSOgF0kQ4/WggKuaY...,None,,,,,CUSTOM,#anonymous-group-id#5dzj2ImJ3mrvZ382mEI1ZrtFyi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046412,#anonymous-user-id#ABmgpvRnyCMeYpZ5VRaoF9/P4P8...,2020-08-27T02:54:18+0000,False,2020-09-04T21:14:53+0000,Ryan,"Hi team,\nCongratulations to all of this years...",#anonymous-user-id#I+zHbaCOl1NUZ5NfWIRxy2Fx0O/...,None,,,,,CUSTOM,#anonymous-group-id#ABmgpvRnyCMeYpZ5VRaoF9/P4P...
2046413,#anonymous-user-id#ABmgpvRnyCMeYpZ5VRaoF9/P4P8...,2020-08-25T06:35:55+0000,False,2020-08-25T06:35:55+0000,NaN,NaN,#anonymous-user-id#6TPbrJ94kwPF4JTiY5e7oR5LWm2...,None,,,,,CUSTOM,#anonymous-group-id#ABmgpvRnyCMeYpZ5VRaoF9/P4P...
2046414,#anonymous-user-id#ABmgpvRnyCMeYpZ5VRaoF9/P4P8...,2020-08-25T04:28:21+0000,False,2020-08-25T04:28:21+0000,NaN,This group will be used to share specific upda...,#anonymous-user-id#6TPbrJ94kwPF4JTiY5e7oR5LWm2...,None,,,,,CUSTOM,#anonymous-group-id#ABmgpvRnyCMeYpZ5VRaoF9/P4P...
2046415,#anonymous-user-id#ABmgpvRnyCMeYpZ5VRaoF9/P4P8...,2020-08-20T06:07:09+0000,False,2020-08-20T06:07:10+0000,NaN,NaN,#anonymous-user-id#6TPbrJ94kwPF4JTiY5e7oR5LWm2...,None,,,,,CUSTOM,#anonymous-group-id#ABmgpvRnyCMeYpZ5VRaoF9/P4P...


In [ ]:
# read all files in input_converted_parquet/group_feeds_*.parquet
# all_group_feeds_gemini = pd.concat([pd.read_parquet(f) for f in glob.glob('input_converted_parquet/group_feeds_*.parquet')])

In [3]:
# all_member_feeds_gemini = pd.concat([pd.read_parquet(f) for f in glob.glob('input_converted_parquet/member_feeds_*.parquet')])
all_conversations_gemini = pd.concat([pd.read_parquet(f) for f in glob.glob('input_converted_parquet/conversations_*.parquet')])
# all_messages_gemini = pd.concat([pd.read_parquet(f) for f in glob.glob('input_converted_parquet/member_messages_*.parquet')])

In [11]:
all_conversations_gemini[all_conversations_gemini['id'] == 't_3045581772126155'].index

Index([    0,     7,    63,    81, 61396, 61635, 61648, 61655, 61786, 61789,
       ...
       44601, 45825, 47466, 47951, 50523, 51119, 52062, 58469, 79202, 81517],
      dtype='int64', length=133)

In [ ]:
conversations_df[conversations_df['id'] == 't_3045581772126155'].index

Index([ 43728,  50558,  65260,  72320,  76076,  77294,  86023,  86655,  87040,
        91119,
       ...
       510098, 524165, 548464, 556444, 566077, 582414, 613835, 615906, 620599,
       641487],
      dtype='int64', length=133)

In [39]:
import pandas as pd
import numpy as np
from collections import defaultdict
import traceback # To print more details on conversion errors


def canonicalize_cell(cell):
    """
    Converts a cell value to a canonical representation for comparison.
    - Handles ndarrays (including empty ones) first.
    - Handles lists explicitly (including empty ones).
    - Handles NaN/None.
    - Handles dicts (including empty ones, recursive canonicalization).
    - Handles other types, ensuring hashability.
    """
    # --- Debugging Start ---
    # Uncomment the line below temporarily to see exactly what's passed in:
    # print(f"  Processing cell: {repr(cell)}, type: {type(cell)}")
    # --- Debugging End ---

    # 1. Check for NumPy array
    if isinstance(cell, np.ndarray):
        if cell.size == 0:
            # print(f"    -> Detected empty ndarray")
            return "np_empty_array" # Distinct representation
        else:
            # print(f"    -> Detected non-empty ndarray")
            try:
                str_elements = []
                for i, e in enumerate(cell):
                    # Recursively canonicalize elements *within* the array
                    canonical_element = canonicalize_cell(e)
                    # Ensure the canonical element is stringified for sorting
                    str_elements.append(str(canonical_element))
                # Return sorted tuple of string representations
                return tuple(sorted(str_elements))
            except Exception as array_proc_error:
                # print(f"Warning: Error processing numpy array: {array_proc_error}")
                try: # Fallback string representation
                    return f"np_array_fallback_{np.array2string(cell, separator=', ')}"
                except:
                    return "np_array_unstringable_fallback"

    # 2. Check for standard Python list (BEFORE pd.isna)
    elif isinstance(cell, list):
        # print(f"    -> Detected list")
        if not cell: # Check if list is empty
            return "py_empty_list" # Distinct representation for empty list
        try:
            str_elements = []
            for i, e in enumerate(cell):
                 # Recursively canonicalize elements *within* the list
                 canonical_element = canonicalize_cell(e)
                 # Ensure the canonical element is stringified for sorting
                 str_elements.append(str(canonical_element))
            # Return sorted tuple of string representations
            return tuple(sorted(str_elements))
        except Exception as list_proc_error:
             # print(f"Warning: Error processing list: {list_proc_error}")
             try: # Fallback string representation
                 return f"py_list_fallback_{str(cell)}"
             except:
                 return "py_list_unstringable_fallback"

    # 3. Check for NaN/None (ONLY if not array or list)
    elif pd.isna(cell):
        # This should now ONLY be reached for scalar NaN/None/NaT values
        # print(f"    -> Detected scalar NaN/None/NaT")
        return "<NA>"

    # 4. Check for dict (if not array, list, or NA)
    elif isinstance(cell, dict):
        # print(f"    -> Detected dict")
        if not cell: # Empty dict
            return "py_empty_dict"
        try:
            # Convert dict to sorted tuple of canonical key:value string pairs
            items_list = []
            for k, v in cell.items():
                # Recursively canonicalize keys and values
                canonical_k = canonicalize_cell(k)
                canonical_v = canonicalize_cell(v)
                items_list.append(f"{str(canonical_k)}:{str(canonical_v)}")
            return tuple(sorted(items_list))
        except Exception as dict_proc_error:
            # print(f"Warning: Error processing dictionary: {dict_proc_error}")
            try: # Fallback string representation
                 return f"dict_fallback_{str(cell)}"
            except:
                 return "dict_unstringable_fallback"

    # 5. Handle other scalar types
    else:
        # print(f"    -> Detected other scalar type: {type(cell)}")
        try:
            # Ensure the value is hashable
            hash(cell)
            return cell
        except TypeError:
            # print(f"Warning: Cell of type {type(cell)} is not hashable. Converting to string.")
            try: # Fallback to string
                return str(cell)
            except: # Ultimate fallback for unstringable objects
                return f"<unstringable_type_{type(cell).__name__}>"

def are_dataframes_equal_detailed(df1: pd.DataFrame, df2: pd.DataFrame) -> bool:
    """
    Checks if two DataFrames are semantically equal, ignoring row/column order
    and handling NaNs, NumPy arrays (with dicts), according to specific rules.

    Args:
        df1: First pandas DataFrame.
        df2: Second pandas DataFrame.

    Returns:
        True if the DataFrames are considered equal, False otherwise.
        Prints the first detected difference if they are not equal.
    """
    print("Starting DataFrame comparison...")

    # --- 1. Initial Checks ---
    if not isinstance(df1, pd.DataFrame) or not isinstance(df2, pd.DataFrame):
        print("Error: Inputs must be pandas DataFrames.")
        return False

    if df1.shape != df2.shape:
        print(f"Difference found: Shapes differ.")
        print(f"  df1 shape: {df1.shape}")
        print(f"  df2 shape: {df2.shape}")
        return False

    if df1.empty: # And df2 is also empty due to shape check
        print("Both DataFrames are empty and have the same shape. Considered equal.")
        return True

    # --- 2. Column Alignment ---
    # Check if column names are the same set, regardless of order
    if set(df1.columns) != set(df2.columns):
        print("Difference found: Column names differ.")
        print(f"  df1 columns: {sorted(list(df1.columns))}")
        print(f"  df2 columns: {sorted(list(df2.columns))}")
        return False

    # Sort columns alphabetically for consistent comparison
    try:
        df1_sorted = df1.sort_index(axis=1)
        df2_sorted = df2.sort_index(axis=1)
        print("Columns sorted alphabetically.")
    except Exception as e:
        print(f"Error sorting columns: {e}. Cannot proceed reliably.")
        return False

    # --- 3. Row Comparison using Canonical Representation ---
    print("Generating canonical representations for rows...")
    df2_canonical_map = defaultdict(list) # Key: canonical_row_tuple, Value: list of original indices
    df2_row_data_map = {} # Key: original index, Value: row Series (for reporting)

# (Inside the are_dataframes_equal_detailed function)

    # Build map for df2
    for idx2, row2 in df2_sorted.iterrows():
        # --- Debugging Specific Row ---
        # Replace 47 with the actual index reported in your error
        TARGET_DEBUG_INDEX = 47
        if idx2 == TARGET_DEBUG_INDEX:
            print(f"\n--- Debugging Row Index {idx2} ---")
            print(f"Raw row data (df2_sorted.loc[{idx2}]):")
            try:
                print(df2_sorted.loc[idx2])
                print("\nRow dtypes:")
                print(df2_sorted.loc[idx2].apply(type))
                print("\nRow values being processed:")
                print(row2.values) # Show the actual array being iterated over
                print("------")
            except Exception as e:
                print(f"Error printing debug info for row {idx2}: {e}")
            print(f"Attempting canonicalization for row {idx2}...")
        # --- End Debugging Specific Row ---

        try:
            # This is the line where the error occurs for row 47
            canonical_row_elements = [canonicalize_cell(cell) for cell in row2.values]
            canonical_row_tuple = tuple(canonical_row_elements)
            df2_canonical_map[canonical_row_tuple].append(idx2)
            if idx2 not in df2_row_data_map:
                df2_row_data_map[idx2] = row2 # Store row data
        except Exception as e:
            print(f"\nError creating canonical representation for df2 row index {idx2}:")
            print(f"  Error: {e}")
            # print(f"  Row data: \n{row2}") # Already printed above if it's the target index
            traceback.print_exc() # Print full traceback for more clues
            print("Cannot reliably compare DataFrames due to this error.")
            return False

# ... rest of the function

    # Match rows from df1 against df2's map
    df1_row_indices = list(df1_sorted.index) # Keep track to find unmatched df1 rows later if needed

    for idx1, row1 in df1_sorted.iterrows():
        try:
            canonical_row_elements = [canonicalize_cell(cell) for cell in row1.values]
            canonical_row_tuple = tuple(canonical_row_elements)
        except Exception as e:
            print(f"\nError creating canonical representation for df1 row index {idx1}:")
            print(f"  Error: {e}")
            print(f"  Row data: \n{row1}")
            # traceback.print_exc() # Optional: Print full traceback
            print("Cannot reliably compare DataFrames due to this error.")
            return False

        # Check if this canonical row exists in df2's map and has available indices
        if canonical_row_tuple in df2_canonical_map and df2_canonical_map[canonical_row_tuple]:
            # Match found: Consume one index from the list for this canonical row
            df2_canonical_map[canonical_row_tuple].pop()
            if idx1 in df1_row_indices: # Should always be true here
                 df1_row_indices.remove(idx1) # Mark df1 row as matched
        else:
            # No match found for this df1 row
            print(f"\nDifference found: Row from df1 (index: {idx1}) has no corresponding match in df2.")
            print("--- Unmatched df1 row (columns sorted) ---")
            try:
                # Try printing Series cleanly
                 print(row1.to_string())
            except:
                 print(row1) # Fallback
            print("-" * 30)
            # You could optionally try to find the "closest" row in df2, but that's complex.
            # Simply stating it's unmatched is usually sufficient.
            return False

    # --- 4. Final Check (Should be unnecessary if shapes match and no errors) ---
    # Check if any counts remain in df2_canonical_map, indicating extra rows in df2 not matched by df1
    remaining_df2_indices = [idx for indices in df2_canonical_map.values() for idx in indices if indices] # Get only non-empty lists flattened
    if remaining_df2_indices:
        first_unmatched_idx2 = remaining_df2_indices[0]
        print(f"\nDifference found: Rows found in df2 that were not matched by any row in df1.")
        print(f"  Example unmatched df2 row (original index: {first_unmatched_idx2}).")
        print("--- Unmatched df2 row (columns sorted) ---")
        try:
            # Try printing Series cleanly
            print(df2_row_data_map[first_unmatched_idx2].to_string())
        except:
            print(df2_row_data_map.get(first_unmatched_idx2, "Error retrieving row data")) # Fallback
        print("-" * 30)
        return False

    # Also check df1_row_indices (should be empty if all matched)
    if df1_row_indices:
        # This case should have been caught in the main loop, but as a safeguard:
        first_unmatched_idx1 = df1_row_indices[0]
        print(f"\nInternal Logic Error: A df1 row (index: {first_unmatched_idx1}) was not matched, but wasn't caught earlier.")
        print("--- Unmatched df1 row (columns sorted) ---")
        try:
             print(df1_sorted.loc[first_unmatched_idx1].to_string())
        except:
             print(df1_sorted.loc[first_unmatched_idx1])
        print("-" * 30)
        return False


    # --- 5. Conclusion ---
    print("\nComparison complete. No differences found.")
    print("DataFrames are considered equal based on the specified criteria.")
    return True


In [45]:
sorted(all_messages_gemini.columns.to_list())

['attachments_data',
 'attachments_paging_cursors_after',
 'attachments_paging_cursors_before',
 'attachments_paging_next',
 'conversation_id',
 'created_time',
 'from_community_id',
 'from_email',
 'from_id',
 'from_name',
 'id',
 'member_id',
 'message',
 'shares_data',
 'shares_paging_cursors_after',
 'shares_paging_cursors_before',
 'shares_paging_next',
 'tag_names']

In [55]:
sorted(all_messages_gemini['conversation_id'].to_list(), reverse=True)

['t_999984690890460',
 't_999984690890460',
 't_999984690890460',
 't_999984690890460',
 't_999984690890460',
 't_999984690890460',
 't_999984690890460',
 't_999908547108029',
 't_999908547108029',
 't_999908547108029',
 't_999908547108029',
 't_999908547108029',
 't_999908547108029',
 't_999908547108029',
 't_999908547108029',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999590407179867',
 't_999499188423293',
 't_999499188423293',
 't_999499188423293',
 't_999099067220544',
 't_999099067220544',
 't_999099067220544',
 't_999099067220544',
 't_999099067220544',
 't_999099067220544',
 't_999099

In [56]:
sorted(all_messages_gemini['conversation_id'].dropna().to_list()) == sorted(messages_df['conversation_id'].dropna().to_list())

True

In [14]:
group_feeds_df

,id,created_time,icon,is_hidden,link,message,object_id,permalink_url,picture,updated_time,...,place_location_state,application_link,application_name,application_id,place_location_located_in,admin_creator_link,admin_creator_name,admin_creator_id,admin_creator_category,application_category
0,1588681094756721_3558254291132715,2024-08-30T01:56:14+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,# **NI Meat Team. Please Read**,518359230882164,https://countdown.facebook.com/groups/15886810...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2024-08-30T01:56:15+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588681094756721_3558187831139361,2024-08-29T23:30:58+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,"All discoloured, do I mark them down or just d...",1967479797019999,https://countdown.facebook.com/groups/15886810...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2024-08-30T06:32:11+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1588681094756721_3558108591147285,2024-08-29T20:34:22+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,Friday and lamb legs still coming in at incorr...,530764715968387,https://countdown.facebook.com/groups/15886810...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2024-08-30T01:39:32+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1588681094756721_3556815604609917,2024-08-28T03:30:03+0000,NaN,False,NaN,Goodafternoon team. Hey I'm having trouble sen...,NaN,https://countdown.facebook.com/groups/15886810...,NaN,2024-08-28T19:00:09+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1588681094756721_3555038108121000,2024-08-26T01:03:56+0000,NaN,False,NaN,Hi Team \n\nOur Hellers Prepacks on Monday see...,NaN,https://countdown.facebook.com/groups/15886810...,NaN,2024-08-26T05:05:40+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390500,934719143328659_947109825422924,2016-12-13T20:55:01+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,Are you an experienced Assistant Long Life Man...,125194934638861,https://countdown.facebook.com/groups/93471914...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2016-12-13T20:55:45+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390501,934719143328659_946533388813901,2016-12-13T02:48:00+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,There's lots of great things about being a par...,226665357771252,https://countdown.facebook.com/groups/93471914...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2017-01-03T22:21:28+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390502,934719143328659_935340133266560,2016-11-29T03:04:55+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,"Hi Team,\n\nDid you know as Countdown team mem...",218748918562896,https://countdown.facebook.com/groups/93471914...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2016-11-29T03:04:55+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390503,934719143328659_935294503271123,2016-11-29T01:11:35+0000,https://www.facebook.com/images/icons/photo.gif,False,https://countdown.workplace.com/photo.php?fbid...,NaN,218695198568268,https://countdown.facebook.com/groups/93471914...,https://scontent-syd2-1.xx.fbcdn.net/v/t39.308...,2016-11-29T01:11:35+0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
all_bunnings_member_feeds_gemini = pd.concat([pd.read_parquet(f) for f in glob.glob('Bunnings_converted_parquet/member_messages_*.parquet')])
all_bunnings_member = pd.read_parquet('Bunnings_converted_parquet/members.parquet')

In [11]:
member_id_in_conversation = set(all_bunnings_member_feeds_gemini['member_id'].unique())
all_member_id = set(all_bunnings_member['id'].unique())

In [12]:
print(f"Number of member IDs in conversation: {len(member_id_in_conversation)}")
print(f"Number of member IDs in all members: {len(all_member_id)}")

Number of member IDs in conversation: 14688
Number of member IDs in all members: 213271


In [14]:
all_bunnings_member_feeds_gemini['from_id'].nunique()

72714

In [17]:
all_bunnings_member_feeds_gemini.shape

(62175638, 10)

In [16]:
print("Number of members in common:", len(member_id_in_conversation.intersection(all_member_id)))

Number of members in common: 14688


In [2]:
all_bunnings_member = pd.read_parquet('Bunnings_converted_parquet/members.parquet')

In [4]:
all_bunnings_member[all_bunnings_member['account_invite_time'].isnull()]['id'].to_list()

['#anonymous-user-id#++1WAzsdgS1C1gqn/Gu24iAbWOIwKCeSGNSKcayvhjo=',
 '#anonymous-user-id#++8g4rCPfllPsku0u/WALPiTryXSdgRuRlWJ5FomGJg=',
 '#anonymous-user-id#++AIZ3i2h9Nl4lVmnDcxRpuroALPuEs8bx6meRWicnY=',
 '#anonymous-user-id#++BiPxwHMg9r98arnGTsUDu5ltdi+gkVfx5m7Rt6EMM=',
 '#anonymous-user-id#++ByHeu6lqozbU4tD4JzIWPUC1TD15Mjwg27HtMTAno=',
 '#anonymous-user-id#++E0yZAvkCDolH8b7go16ZVBxh3dpHAMmIkRwq7W4SE=',
 '#anonymous-user-id#++GiIZKCkmSGo0QcL4YeoBoeiNGorJwy7loH5AqCvas=',
 '#anonymous-user-id#++H1A8iUFUOPqFGB4qwy9XDN2pLRWUx2VsIAQBqgK7A=',
 '#anonymous-user-id#++IvrYVx888cCm7JAvM9ZAvvvLjcJoGHsfue7vuEKik=',
 '#anonymous-user-id#++MUXiNqtOPwzV3LA465VM1vtdAo+yJiI2sscjQz99w=',
 '#anonymous-user-id#++Zf5kq5lOn5QIXkZibPiggqpxe4Ri/5PJqlN6DkANw=',
 '#anonymous-user-id#++bg5BnUyiGxRrPU/1lqczK2YBU0aiPNBYJK0ZLWuFo=',
 '#anonymous-user-id#++lVpfcIuVyi7RCPssQLyjqLSQhYl+uTn8py+fQ8IgQ=',
 '#anonymous-user-id#++r/14dA6ExCPjtspVdbTe7oTODNjXOUZe7SSvVzNNo=',
 '#anonymous-user-id#++wK+Y9IHDceBPzZcuoNE1cvFdz

In [25]:
all_bunnings_member.shape

(213271, 13)

In [10]:
with open("input/post-data/post-summaries/data-part1.json") as f:
    post_summaries = json.load(f)

post_summaries_df = pd.DataFrame()

for post_id, post_content in post_summaries.items():
    post = pd.json_normalize(post_content, sep='_')
    post['post_id'] = post_id
    post_summaries_df = pd.concat([post_summaries_df, post], ignore_index=True)

In [11]:
post_summaries_df

,seen,comments_order,comments_total_count,comments_can_comment,reactions_total_count,reactions_viewer_reaction,post_id,seen_total_count
0,None,chronological,0,False,0,NONE,413061281065947_413061294399279,NaN
1,None,chronological,0,False,0,NONE,1486438495467759_1486438512134424,1.0
2,None,chronological,0,False,0,NONE,200367026406960_200367043073625,1.0
3,None,chronological,0,False,0,NONE,1472025353519797_1472025393519793,2.0
4,None,chronological,0,False,0,NONE,413061281065947_413061311065944,1.0
...,...,...,...,...,...,...,...,...
174,None,chronological,3,False,26,NONE,2328007430742432_2338327893043719,1978.0
175,None,chronological,0,False,0,NONE,763160699001955_763160732335285,NaN
176,None,chronological,9,False,16,NONE,2328007430742432_2352679984941843,2444.0
177,None,chronological,10,False,32,NONE,2328007430742432_2361601140716394,3330.0
